In [1]:
pip install d2l

##**Library**

In [1]:
import torch
from torch import nn
import d2l.torch as d2l

##**Dense Block**

In [2]:
def conv_block(num_channels):
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.LazyConv2d(num_channels, kernel_size=3, padding=1))

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, num_channels):
        super(DenseBlock, self).__init__()
        layer = []
        for i in range(num_convs):
            layer.append(conv_block(num_channels))
        self.net = nn.Sequential(*layer)

    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            X = torch.cat((X, Y), dim=1)
        return X

In [4]:
blk = DenseBlock(2, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/lazy.py:181: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([4, 23, 8, 8])

##**Tansition Layer**

In [5]:
def transition_block(num_channels):
    return nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.ReLU(), #  Layer aktivasi ReLU
        nn.LazyConv2d(num_channels, kernel_size=1),
        nn.AvgPool2d(kernel_size=2, stride=2))

In [6]:
blk = transition_block(10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

In [7]:
class DenseNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3),
            nn.LazyBatchNorm2d(), nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [8]:
@d2l.add_to_class(DenseNet)
def __init__(self, num_channels=64, growth_rate=32, arch=(4, 4, 4, 4),
             lr=0.1, num_classes=10):
    super(DenseNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1())
    for i, num_convs in enumerate(arch):
        self.net.add_module(f'dense_blk{i+1}', DenseBlock(num_convs,
                                                          growth_rate))

        num_channels += num_convs * growth_rate
        if i != len(arch) - 1:
            num_channels //= 2
            self.net.add_module(f'tran_blk{i+1}', transition_block(
                num_channels))
    self.net.add_module('last', nn.Sequential(
        nn.LazyBatchNorm2d(), nn.ReLU(),
        nn.AdaptiveAvgPool2d((1, 1)), nn.Flatten(),
        nn.LazyLinear(num_classes)))
    self.net.apply(d2l.init_cnn)

##**Training**

In [ ]:
model = DenseNet(lr=0.01)
trainer = d2l.Trainer(max_epochs=10, num_gpus=1)
data = d2l.FashionMNIST(batch_size=128, resize=(96, 96))
trainer.fit(model, data)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


##**Penjelasan**

implementasi dari DenseNet (Densely Connected Convolutional Networks) menggunakan PyTorch dan modul d2l (d2l.torch). DenseNet adalah arsitektur jaringan saraf konvolusional (CNN) yang terkenal karena konektivitas yang padat antara layer-layernya. Berikut penjelasan untuk setiap bagian codingan tersebut:


1.   Import Library:
*   import torch: Mengimpor library PyTorch untuk komputasi tensor.
*   from torch import nn: Mengimpor komponen jaringan saraf seperti lapisan (layers).
*   import d2l.torch as d2l: Mengimpor modul d2l.torch yang menyediakan utilitas dan fungsi tambahan untuk deep learning dengan PyTorch.

2.   Definisi Convolutional Block (conv_block):
*   conv_block(num_channels): Fungsi untuk membuat blok konvolusi, termasuk Batch Normalization, ReLU, dan Convolutional layer dengan kernel size 3x3 dan padding 1.
3. Dense Block (DenseBlock):
*   Kelas DenseBlock: Kelas turunan dari nn.Module yang mendefinisikan blok dense dalam DenseNet.
*   __init__: Konstruktor untuk DenseBlock, menerima jumlah konvolusi (num_convs) dan jumlah channel (num_channels), membuat num_convs blok konvolusi menggunakan conv_block.
*   forward: Metode untuk propagasi maju, melakukan iterasi melalui setiap blok dalam self.net dan menggabungkan outputnya dengan input menggunakan concatenation.
4. Transition Block (transition_block):
*   transition_block(num_channels): Fungsi untuk membuat blok transisi, termasuk Batch Normalization, ReLU, Convolutional layer dengan kernel size 1x1, dan Average Pooling dengan kernel size 2x2 dan stride 2.
5. DenseNet (DenseNet):
*   Kelas DenseNet: Kelas turunan dari d2l.Classifier untuk implementasi DenseNet.
*   b1: Metode untuk lapisan pertama DenseNet, termasuk Convolutional layer, Batch Normalization, ReLU, dan Max Pooling.
*  __init__: Konstruktor DenseNet, menerima beberapa parameter seperti num_channels, growth_rate, arch (arsitektur DenseNet), lr (learning rate), dan num_classes (jumlah kelas).
  * Inisialisasi self.net sebagai nn.Sequential dengan b1 sebagai lapisan pertama.
  * Iterasi melalui arch untuk menambahkan DenseBlock dan transition_block ke self.net, menyesuaikan num_channels berdasarkan growth_rate.
  * Menambahkan lapisan akhir dengan Batch Normalization, ReLU, Adaptive Average Pooling, Flatten, dan Linear layer untuk klasifikasi.
  * Mengaplikasikan inisialisasi jaringan (d2l.init_cnn) ke semua lapisan dalam self.net.
6. Instansiasi Model, Trainer, dan Dataset:
* model = DenseNet(lr=0.01): Membuat instance DenseNet dengan learning rate 0.01.
* trainer = d2l.Trainer(max_epochs=10, num_gpus=1): Membuat instance Trainer dengan maksimal 10 epoch dan menggunakan 1 GPU.
* data = d2l.FashionMNIST(batch_size=128, resize=(96, 96)): Memuat dataset FashionMNIST dengan batch size 128 dan ukuran gambar yang diresize menjadi 96x96 piksel.
7. Pelatihan Model:
* trainer.fit(model, data): Melatih model DenseNet menggunakan dataset FashionMNIST dengan menggunakan Trainer yang sudah didefinisikan sebelumnya.


Ini adalah implementasi lengkap dari DenseNet menggunakan PyTorch dengan menggunakan modul d2l untuk bantuan tambahan dalam proses pelatihan dan evaluasi model.